In [ ]:
# Active: 1767925344562@@127.0.0.1@5432
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TesteVSCode").getOrCreate()
print("Conectado ao Spark no Docker. Versão:", spark.version)


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/01/10 00:35:09 WARN Utils: Your hostname, Emivalto, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/10 00:35:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/10 00:35:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Conectado ao Spark no Docker. Versão: 4.1.1


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when

# 1. Inicializa a Sessão Spark (Necessário para rodar o código)
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("ETL_Enem_Bronze_Silver") \
    .config("spark.jars", "/path/to/postgresql-42.2.18.jar") \
    .getOrCreate() 
    # Nota: Você precisará baixar o driver JDBC do Postgres (.jar) se for gravar no banco real

# ==============================================================================
# ETAPA 1: EXTRACT (Ler da Bronze)
# ==============================================================================
# O arquivo ENEM usa separador ';' e codificação 'latin1' (padrão Brasil)
caminho_arquivo = "/home/emivalto/workspace/BD2V/SDBD2---INEP/raw/MICRODADOS_ENEM_2021.csv" # Ajuste o caminho se estiver em subpasta

df_bronze = spark.read \
    .option("header", "true") \
    .option("delimiter", ";") \
    .option("encoding", "ISO-8859-1") \
    .option("inferSchema", "true") \
    .csv(caminho_arquivo)

print("Quantidade de registros brutos:", df_bronze.count())

# ==============================================================================
# ETAPA 2: TRANSFORM (Limpeza e Padronização)
# ==============================================================================

# Selecionar apenas colunas úteis e renomear para algo legível (boas práticas)
df_silver = df_bronze.select(
    col("NU_INSCRICAO").alias("id_inscricao"),
    col("NU_ANO").alias("ano"),
    col("TP_SEXO").alias("sexo"),
    col("NO_MUNICIPIO_PROVA").alias("cidade_prova"),
    col("SG_UF_PROVA").alias("uf_prova"),
    col("NU_NOTA_CN").cast("float").alias("nota_ciencias_natureza"),
    col("NU_NOTA_CH").cast("float").alias("nota_humanas"),
    col("NU_NOTA_LC").cast("float").alias("nota_linguagens"),
    col("NU_NOTA_MT").cast("float").alias("nota_matematica"),
    col("NU_NOTA_REDACAO").cast("float").alias("nota_redacao")
)

# Tratamento: Preencher notas nulas com 0 (quem faltou fica com zero na nossa regra)
df_silver = df_silver.fillna(0, subset=[
    "nota_ciencias_natureza", "nota_humanas", "nota_linguagens", "nota_matematica", "nota_redacao"
])

# Mostra como ficou a tabela tratada
print("Amostra dos dados tratados (Silver):")
df_silver.show(5)

# ==============================================================================
# ETAPA 3: LOAD (Gravar na Silver / Banco de Dados)
# ==============================================================================

# Configurações do seu banco Postgres (que está no Docker)
db_url = "jdbc:postgresql://localhost:5432/nome_do_seu_banco"
db_properties = {
    "user": "seu_usuario",       # Geralmente 'postgres'
    "password": "sua_senha",     # A senha que você definiu no docker-compose
    "driver": "org.postgresql.Driver"
}

# Comenta esta linha abaixo se você ainda não subiu o Postgres, para não dar erro
# df_silver.write.jdbc(url=db_url, table="silver_enem", mode="overwrite", properties=db_properties)

print("Processo finalizado com sucesso!")

Quantidade de registros brutos: 3389832
Amostra dos dados tratados (Silver):
+------------+----+----+--------------------+--------+----------------------+------------+---------------+---------------+------------+
|id_inscricao| ano|sexo|        cidade_prova|uf_prova|nota_ciencias_natureza|nota_humanas|nota_linguagens|nota_matematica|nota_redacao|
+------------+----+----+--------------------+--------+----------------------+------------+---------------+---------------+------------+
|210053865474|2021|   F|           Nova Lima|      MG|                   0.0|       574.6|          472.6|            0.0|       760.0|
|210052384164|2021|   M|              Maceió|      AL|                 505.9|       551.8|          498.3|          461.5|       560.0|
|210052589243|2021|   F|Ferraz de Vasconc...|      SP|                   0.0|         0.0|            0.0|            0.0|         0.0|
|210052128335|2021|   M|               Crato|      CE|                 580.7|       678.9|          638.9| 